In [ ]:
import zipfile
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.neighbors import NearestNeighbors


In [ ]:
import zipfile
import pandas as pd
import json

api_token={"username":"luisfhidalgo",
           "key":"d4e4e6425d85ef204447086541cb5b11"}
!rm -rf /roo/.kaggle. # Elimina de forma recursiva y forzada un directorio y su contenido.Por si existe
!mkdir  /root/.kaggle # Crea Directorio

with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(api_token, file)
!chmod 600 /root/.kaggle/kaggle.json
!kaggle datasets download -d zynicide/wine-reviews
archivo_zip = 'wine-reviews.zip'

# Listar los archivos dentro del ZIP
with zipfile.ZipFile(archivo_zip, 'r') as zip_file:
    archivos = zip_file.namelist()
    print("Archivos en el ZIP:")
    for archivo in archivos:
        print(archivo)
#Cargo los Archivos Dataset a los dataframe que voy a utilizar

archivo_zip = 'wine-reviews.zip'

with zipfile.ZipFile(archivo_zip, 'r') as zip_file:

        with zip_file.open('winemag-data_first150k.csv') as winemag_150_csv:
            winemag_df = pd.read_csv(winemag_150_csv)


mkdir: cannot create directory ‘/root/.kaggle’: File exists
Dataset URL: https://www.kaggle.com/datasets/zynicide/wine-reviews
License(s): CC-BY-NC-SA-4.0
wine-reviews.zip: Skipping, found more recently modified local copy (use --force to force download)
Archivos en el ZIP:
winemag-data-130k-v2.csv
winemag-data-130k-v2.json
winemag-data_first150k.csv


In [ ]:
winemag_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150930 entries, 0 to 150929
Data columns (total 11 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   Unnamed: 0   150930 non-null  int64  
 1   country      150925 non-null  object 
 2   description  150930 non-null  object 
 3   designation  105195 non-null  object 
 4   points       150930 non-null  int64  
 5   price        137235 non-null  float64
 6   province     150925 non-null  object 
 7   region_1     125870 non-null  object 
 8   region_2     60953 non-null   object 
 9   variety      150930 non-null  object 
 10  winery       150930 non-null  object 
dtypes: float64(1), int64(2), object(8)
memory usage: 12.7+ MB


In [ ]:
duplicados = winemag_df.duplicated().sum()
print("Cantidad de duplicados:",duplicados) # son registros exactamente iguales
# Busca registros que contengan la frase
registros_coincidentes = winemag_df[winemag_df['description'].str.contains("This standout Rocks District wine brings earth")]
# Imprime los registros coincidentes
print(registros_coincidentes)
winemag_df.drop_duplicates(inplace=True)
# Se elimina columna  'Unnamed: 0',
winemag_df.drop('Unnamed: 0', axis=1, inplace=True)
winemag_df.isnull().sum()
# Se completan valores nulos en 'designation' y 'region_1'
winemag_df.loc[:, 'designation'] = winemag_df['designation'].fillna('Unknown')
winemag_df.loc[:, 'region_1'] = winemag_df['region_1'].fillna('Unknown')
winemag_df.loc[:, 'region_2'] = winemag_df['region_2'].fillna('Unknown')
# Imputación con la mediana en el campo 'price'
mediana_precio = winemag_df['price'].median()
winemag_df['price'] = winemag_df['price'].fillna(mediana_precio)
# Se elimina  las 5 filas con valores nulos en la columna 'country'
winemag_df.dropna(subset=['country'], inplace=True)
df= winemag_df.copy()

Cantidad de duplicados: 0
     Unnamed: 0 country                                        description  \
29           29      US  This standout Rocks District wine brings earth...   
300         300      US  This standout Rocks District wine brings earth...   

         designation  points  price    province                 region_1  \
29   The Funk Estate      94   60.0  Washington  Walla Walla Valley (WA)   
300  The Funk Estate      94   60.0  Washington  Walla Walla Valley (WA)   

            region_2 variety  winery  
29   Columbia Valley   Syrah  Saviah  
300  Columbia Valley   Syrah  Saviah  


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 150925 entries, 0 to 150929
Data columns (total 10 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   country      150925 non-null  object 
 1   description  150925 non-null  object 
 2   designation  150925 non-null  object 
 3   points       150925 non-null  int64  
 4   price        150925 non-null  float64
 5   province     150925 non-null  object 
 6   region_1     150925 non-null  object 
 7   region_2     150925 non-null  object 
 8   variety      150925 non-null  object 
 9   winery       150925 non-null  object 
dtypes: float64(1), int64(1), object(8)
memory usage: 12.7+ MB


In [ ]:
# Seleccionar columnas relevantes
df = df[['variety', 'country', 'description', 'points', 'price', 'winery']]
df.dropna(subset=['variety', 'country', 'description', 'points', 'winery'], inplace=True)
df.fillna({'price': df['price'].median()}, inplace=True)

# Modelo basado en contenido
vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = vectorizer.fit_transform(df['description'])
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

def recommend_wines_based_on_content(winery, num_recommendations=5):
    if winery not in df['winery'].values:
        return "Bodega no encontrada en la base de datos."
    idx = df[df['winery'] == winery].index[0]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:num_recommendations+1]
    wine_indices = [i[0] for i in sim_scores]
    return df.iloc[wine_indices][['variety', 'country', 'points', 'price', 'winery']]

# Modelo colaborativo
user_item_matrix = df.pivot_table(index='variety', columns='winery', values='points').fillna(0)
model_knn = NearestNeighbors(metric='cosine', algorithm='brute')
model_knn.fit(user_item_matrix)

def recommend_wines_based_on_users(variety, num_recommendations=5):
    if variety not in user_item_matrix.index:
        return "Variedad no encontrada en la base de datos."
    distances, indices = model_knn.kneighbors([user_item_matrix.loc[variety]], n_neighbors=num_recommendations+1)
    wine_varieties = [user_item_matrix.index[i] for i in indices.flatten()[1:]]
    return df[df['variety'].isin(wine_varieties)][['variety', 'country', 'points', 'price', 'winery']].drop_duplicates()

# Ejemplo de uso
print(recommend_wines_based_on_content("Chateau Ste. Michelle"))
print(recommend_wines_based_on_users("Riesling"))


<ipython-input-17-75f852d42f66>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(subset=['variety', 'country', 'description', 'points', 'winery'], inplace=True)
<ipython-input-17-75f852d42f66>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.fillna({'price': df['price'].median()}, inplace=True)
